In [3]:
%pylab inline
from state.map import X_MAX, Y_MAX, ARENA_WALLS 
from math import sqrt
from math import pi
import state.particle_filtering
import numpy as np

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [ ]:
# Generate empty space with right dimensions

def gen_lines():
    lines = []
    for wall in ARENA_WALLS:
        start = wall[0]
        delta = wall[1]

        end = np.add(start, delta)

        line_xs = []
        line_ys = []

        line_xs.append(start[0])
        line_xs.append(end[0])
        line_ys.append(start[1])
        line_ys.append(end[1])

        lines.append(Line2D(line_xs, line_ys, linewidth=2))
        
    return lines 

def plot(particles, robot):
    dpi = 80
    width = X_MAX / dpi;
    height = Y_MAX / dpi;
    figure = plt.figure(figsize=(width,height))

    ax = figure.add_subplot(1,1,1)

    ax.set_ylim(top=Y_MAX, bottom=0)
    ax.set_xlim(right=X_MAX, left=0)

    lines = gen_lines()
    for line in lines:
        ax.add_line(line)

    transpose_locations = particles.locations.T
    transpose_orientations = particles.orientations.T
    
    ax.quiver(transpose_locations[0], transpose_locations[1], np.sin(transpose_orientations), np.cos(transpose_orientations), pivot='mid')
    
    x, y, o = particles.get_position_by_weight(position_confidence=False)
    ax.quiver(x, y, np.sin(o), np.cos(o), color='r', pivot='mid')
    ax.quiver(robot.location[0], robot.location[1], 
              np.sin(robot.orientation), np.cos(robot.orientation), 
              color='b', pivot='mid')

    return figure

In [ ]:
figure = plot(particles, robot)
plt.show(figure)

In [ ]:
reload(state.particle_filtering)
particles = state.particle_filtering.Particles(n=1000, where='1base')
robot = state.particle_filtering.Robot(30, 40, 0)

In [ ]:
particles.resample()

In [ ]:
for i in xrange(1):
    factor = -10
    robot.rotate(pi/factor)
    particles.rotate(pi/factor)
    particles.sense(robot.measurement_prediction())
    distance = 10.0
    robot.forward(distance)
    particles.forward(distance)
    particles.sense(robot.measurement_prediction())
    figure = plot(particles, robot)
    plt.show(figure)
    particles.resample()

In [ ]:
robot.measurement_prediction()

In [ ]:
particles.resample()

In [ ]:
robot.measurement_prediction()

In [ ]:
robot.rotate(1)

In [ ]:
reload(state.particle_filtering)

In [ ]:
particles.resample()

In [ ]:
orientations = np.zeros(10)
ROTATION_STD_ABS = (5.0/360.0)*2*pi
rotation = 1

In [ ]:
%timeit (orientations + (np.random.rand(10) * ROTATION_STD_ABS - 0.5 * ROTATION_STD_ABS + rotation)) % (2.0 * pi)

In [ ]:
%timeit np.mod(np.add(orientations,np.add(np.multiply(np.random.rand(10),ROTATION_STD_ABS),-0.5*ROTATION_STD_ABS + rotation)), 2.0 * pi)

In [ ]:
np.array([
            [ np.cos(orientations), np.sin(orientations)],
            [-np.sin(orientations), np.cos(orientations)]
        ])

In [ ]:
N = 10

In [ ]:
unit = np.concatenate([np.zeros((N,1)), np.ones((N,1))], axis=1)

In [ ]:
FORWARD_STD_FRAC = 0.01

In [ ]:
np.add(
    np.multiply(
        np.random.rand(N),
        FORWARD_STD_FRAC*distance
    ),
    (1-0.5*FORWARD_STD_FRAC)*distance
)

In [ ]:
reload(state.particle_filtering)

In [ ]:
a = state.particle_filtering.Particles.generate_raycasting_distances()

In [ ]:
data_10_20 = np.load('data_10_20.dat.npy')
data_20_30 = np.load('data_20_30.dat.npy')
data_30_40 = np.load('data_30_40.dat.npy')
raycasting_distances = np.concatenate([data_00_10[0:10],data_10_20[10:20],data_20_30[20:30],data_30_40[30:40]])
np.save("raycasting_distances", raycasting_distances)

In [ ]:
data = []
for i in xrange(8):
    data.append(np.load('bin2/data_' + str(i*20) + '_' + str((i+1)*20) + '.dat.npy')[i*20:(i+1)*20])
raycasting_distances_bin2 = np.concatenate(data)
raycasting_distances_bin2

In [ ]:
np.save("raycasting_distances_bin2", raycasting_distances_bin2)

In [ ]:
raycasting_distances[3][3][126]

In [ ]:
reload(state.particle_filtering)

In [ ]:
n=100000
distance = 60
measurements = state.particle_filtering.Particles.model_beam(distance, n)
predictions = np.multiply(np.ones(n), distance)

In [ ]:
state.particle_filtering.Particles.learn_intrinsic_parameters(measurements, predictions)

In [ ]:

np.std(measurements)

In [4]:
a = []
f = open('./log/collect_front_IR_and_sonar_measurements0.csv', 'r')
line = f.readline()
while line:
    line = line.replace("\n", "")
    line = line.replace("[", "")
    line = line.replace("]", "")
    line = line.split(",")
    b=[]
    for i in line:
        b.append(float(i))
    a.append(b)
    line = f.readline()

measurements = np.array(a)

In [5]:
measurements.T[0].shape

(53206,)

In [36]:
predictions = np.multiply(59.0, np.ones(measurements.T[0].shape))

In [35]:
predictions120 = np.multiply(120.0, np.ones(measurements.T[0].shape))

In [10]:
predictions.shape

(53206,)

In [38]:
import robot.state.particle_filtering
reload(robot.state.particle_filtering)

<module 'robot.state.particle_filtering' from 'robot/state/particle_filtering.py'>

In [39]:
robot.state.particle_filtering.Particles.learn_intrinsic_parameters(measurements.T[0], predictions120)

[  9.56000109e-01   1.40907295e-06   1.02753333e+00   4.73875483e-01]
[ 1.47533728  0.25893952  0.2528996   0.47023399]
[ 1.13612821  0.6837292   0.16973938  0.46781351]
[ 1.12777724  0.69337426  0.16849667  0.46776212]
[ 1.1276737   0.69348846  0.16848623  0.46776192]
[ 1.12767239  0.69348941  0.16848658  0.46776198]
[ 1.12767235  0.69348938  0.16848664  0.46776198]
[ 1.12767234  0.69348936  0.16848664  0.46776198]
[ 1.12767233  0.69348936  0.16848664  0.46776198]
[ 1.12767233  0.69348936  0.16848664  0.46776198]


[1.1276723335289058,
 0.6934893607396542,
 0.1684866409051181,
 0.4677619782207791,
 25.653927530165166,
 0.010901277645409909]

In [40]:
robot.state.particle_filtering.Particles.learn_intrinsic_parameters(measurements.T[1], predictions)

[  3.81243528e+00   1.54740715e-06   1.93934239e-01   0.00000000e+00]
[ 3.697511    0.19471828  0.11414156  0.        ]
[ 3.51395554  0.38106169  0.11135351  0.        ]
[ 3.51323915  0.3817866   0.1113452   0.        ]
[ 3.51323783  0.38178773  0.11134521  0.        ]
[ 3.51323782  0.38178774  0.11134521  0.        ]
[ 3.51323782  0.38178774  0.11134521  0.        ]
[ 3.51323782  0.38178774  0.11134521  0.        ]
[ 3.51323782  0.38178774  0.11134521  0.        ]
[ 3.51323782  0.38178774  0.11134521  0.        ]


[3.5132378215960895,
 0.3817877358033755,
 0.11134520952984188,
 0.0,
 2.5689187956017063,
 0.017801905399325038]

In [41]:
robot.state.particle_filtering.Particles.learn_intrinsic_parameters(measurements.T[2], predictions)

[  3.58907791e+00   5.54015792e-07   1.72048649e-01   0.00000000e+00]
[ 3.35923778  0.29252314  0.10936599  0.        ]
[ 3.19752347  0.4569699   0.10663376  0.        ]
[ 3.19647098  0.45801365  0.10664223  0.        ]
[ 3.19646645  0.45801761  0.10664302  0.        ]
[ 3.19646638  0.45801763  0.10664304  0.        ]
[ 3.19646638  0.45801763  0.10664304  0.        ]
[ 3.19646638  0.45801763  0.10664304  0.        ]
[ 3.19646638  0.45801763  0.10664304  0.        ]
[ 3.19646638  0.45801763  0.10664304  0.        ]


[3.1964663753394977,
 0.45801763386224276,
 0.1066430442794567,
 0.0,
 3.3367205675817155,
 0.01813218480166167]